In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

## Connect to EXACT

In [4]:
configuration = Configuration()
configuration.username = 'marzahl' #'exact'
configuration.password = '####' #'exact'
configuration.host = "https://exact.cs.fau.de" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [5]:
annotation_types_points = {}

for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.id] = annotation_type

annotation_types_points

{384: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 384,
  'name': 'L0',
  'node_count': 0,
  'product': 104,
  'sort_order': 0,
  'vector_type': 1},
 385: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 385,
  'name': 'L1',
  'node_count': 0,
  'product': 104,
  'sort_order': 1,
  'vector_type': 1},
 386: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 386,
  'name': 'L2',
  'node_count': 0,
  'product': 104,
  'sort_order': 2,
  'vector_type': 1},
 387: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,

In [13]:
annotations = []

for image_set_name in tqdm(["CCMCT_2.0HT", "CCMCT_Aperio", "CCMCT_Axio", "CCMCT_S210", 
                       "Cyto_2.0HT", "Cyto_Aperio", "Cyto_Axio", "Cyto_S210"]):
    
    scanner = image_set_name.split("_")[1]
    image_set = image_sets_api.list_image_sets(name=image_set_name, expand="product_set,product_set.annotationtype_set").results[0]

    for image_id in image_set.images:

        image = images_api.retrieve_image(id=image_id)
        
        image_width, image_height = image.width, image.height

        for anno in annotations_api.list_annotations(image=image_id, pagination=False, # BoundBox ID
                                                     deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:

            image_type = "CCMCT" if "CCMCT" in image_set_name else "Cyto"
            
            if anno.annotation_type in annotation_types_points:
            
                anno.annotation_type = annotation_types_points[anno.annotation_type]
                annotations.append([scanner, image.id, image.name, image_type, image_width, image_height, 
                                    anno.id, anno.vector, anno.unique_identifier, anno.annotation_type.id, 
                                    anno.annotation_type.name])


annotations = pd.DataFrame(annotations, columns=["scanner", "image_id", "image_name", "image_type", "image_width", "image_height",
                                                 "id", "vector", "unique_identifier", "annotation_type", "type_name"])
annotations.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:18<00:00,  2.29s/it]


,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,type_name
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755406,"{'x1': 20354, 'x2': 20408, 'y1': 16752, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,L0
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755407,"{'x1': 20256, 'x2': 20310, 'y1': 35448, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,L1
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755408,"{'x1': 20445, 'x2': 20499, 'y1': 54139, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,L2
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755409,"{'x1': 26495, 'x2': 26549, 'y1': 71854, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,L3
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755410,"{'x1': 35443, 'x2': 35497, 'y1': 90091, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,L4


In [14]:
annotations.to_csv("GT.csv", index=False)